This script fetches stock data for a user-defined ticker symbol, visualizes it as a candlestick chart,
and uses OpenAI's API to analyze the chart.

In [1]:
# Importing the necessary libraries
import pandas as pd
import plotly.graph_objects as go
import requests
from openai import OpenAI
import base64
import os
from dotenv import load_dotenv

In [7]:
# Load environment variables
load_dotenv()

av_api_key = os.getenv("AI_KEY")
oa_api_key = os.getenv("OA_KEY")
chart_path = os.getenv("CHART_PATH")

In [12]:
# Function to fetch daily stock data from Alpha Vantage API
def fetch_stock_data(ticker):
    global av_api_key
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=compact&apikey={av_api_key}&datatype=csv"
    
    try:
        r = requests.get(url)
        r.raise_for_status()

        return r 
    
    except requests.exceptions.HTTPError as err:
        print(f'HTTP error occurred: {err}')
        return None

ticker = None

# Main function to prompt user for ticker symbol and fetch stock data
def main():

    global ticker 
    ticker = input("Enter a stock ticker: ").upper()

    stock_data = fetch_stock_data(ticker)

    if stock_data:
        with open(f'{ticker}_data.csv', 'wb') as file:
            file.write(stock_data.content)
    else:
        print("No data was fetched.")

main()

In [13]:
# Load stock data from the CSV file for visualization
df = pd.read_csv(f'{ticker}_data.csv')

# Charting the stock data using Plotly
fig = go.Figure(data = [go.Candlestick(x = df['timestamp'],
                open = df['open'],
                close = df['close'],
                high = df['high'],
                low = df['low'])])

# Update the layout of the chart
fig.update_layout(
    title = f'{ticker} Stock Price',
    xaxis_title = 'Date',
    yaxis_title = 'Price (USD)',
    template = 'plotly_dark'
)

# Display the chart
fig.show()

In [15]:
client = OpenAI(api_key = oa_api_key)

# Encode the candlestick chart image as base64 for embedding in API request
with open(chart_path, 'rb') as file:
    base64_image = base64.b64encode(file.read()).decode('utf-8')

# Send the chart and prompt to OpenAI's model for analysis
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {
            "role": "system", "content": "You are a stock trader. You are looking at a candlestick chart of a stock.",
            "role": "user", "content": [
                {
                    "type": "text", 
                    "text": "Analyse the chart, find the support and ressistance levels, look for candlestick patterns, and tell me if I should go long or short on the stock."
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_image}"
                    }
                } 
            ]
        }
    ],
    max_tokens = 500,
)

# Print the AI's response to the analysis prompt
print(response.choices[0].message.content)

To analyze the stock chart for DJT, let's discuss the potential support and resistance levels, candlestick patterns, and implications for trading decisions:

### Support and Resistance Levels
1. **Support Levels**: 
   - **Around 20 USD**: The stock appears to have bounced off this level multiple times, indicating it might act as a strong support area.
   - **Closer to 30 USD**: A consolidation pattern suggests that the stock may find support around this level, as it has held firm during minor downtrends.

2. **Resistance Levels**:
   - **Around 40 USD**: There is a visible resistance at this level, where the stock has faced rejection multiple times.
   - **Mid-30s USD Range**: This area also seems to serve as a resistance point during the consolidating phase.

### Candlestick Patterns
- **Recent Price Action**: The chart shows a bullish trend in the latter part, with green candles indicating upward movement. The last few weeks feature a series of green candles, which might suggest con